In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp dialog_system

In [ ]:
# export
from let_me_answer_for_you.settings import *
import logging
from collections import defaultdict

logging.basicConfig(
    #filename='example.log',
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.ERROR,
    datefmt='%I:%M:%S'
)

logging.debug(" Debug Log Active")
logging.info("Hello! Welcome to our automated dialog system!")
logging.warning(' Warning Log Active')
logging.error(' Error Log Active ')

03:53:49 ERROR: Error Log Active 


In [ ]:
#export
class DialogSystem:
    def __init__(
        self, context_data_file=None, faq_data_file=None, configs_faq=None
    ):
        run_shell_installs()
        self.data = {'context': defaultdict(str), 'faq': defaultdict(str)}

        load_and_prepare_data(
            context_data_file=context_data_file,
            faq_data_file=faq_data_file,
            configs_faq=configs_faq,
            data=self.data
        )
        self.qa_models = load_qa_models(config_tfidf=self.data['faq']['config'])

    def question_answer(self):
        question, responses = question_response(
            data=self.data,
            qa_models=self.qa_models,
            num_returned_values_per_squad_model=1
        )
        print('\n\n'+responses)

    def new_answer(self):
        ...

    def new_context(self):
        ...

In [ ]:
question_response??

Signature: question_response(data, qa_models, num_returned_values_per_squad_model=1)
Docstring: <no docstring>
Source:   
def question_response(data, qa_models, num_returned_values_per_squad_model=1):
    question = get_input('Introduce question:\n')

    _, formatted_responses = get_responses(
        data['context']['df'], question, qa_models, nb_squad_results=2
    )

    return question, formatted_responses
File:      /wd/nbs/let_me_answer_for_you/settings.py
Type:      function


In [ ]:
ds =  DialogSystem()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
100%|██████████| 12.3k/12.3k [00:00<00:00, 3.69MB/s]
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence 

In [ ]:
ds.question_answer()

Introduce question:
 How you doing?




How you doing?:

0: Elon Musk is the CEO of Tesla
1: car company left in california



In [ ]:
#tests
import tempfile
from unittest.mock import patch
from shutil import copyfile
from collections import defaultdict

def mock_faq_files(tmpdirname, faqs, faq_dic):

    faq_dic['path'] = path.join(tmpdirname, 'temp_faq.csv')
    faq_dic['config'] = path.join(tmpdirname, 'temp_config_faq.json')
    faq_dic['df'] = pd.DataFrame(faqs)
    faq_dic['df'].to_csv(faq_dic['path'], index=False)

    copyfile(configs.faq.tfidf_logreg_en_faq, faq_dic['config'])

    updates_faq_config_file(
        configs_path=faq_dic['config'],
        dataset_reader={'data_path': faq_dic['path']}
    )


def mock_context_file(tmpdirname, contexts, context_dic):

    context_dic['path'] = path.join(tmpdirname, 'temp_context.csv')
    context_dic['df'] = pd.DataFrame(contexts)
    context_dic['df'].to_csv(context_dic['path'], index=False)

@patch('__main__.get_input')
def test_updated_faq_response(mock_input):

    new_answer = 'Intekglobal is one of the best companies in the world'
    question = 'What is Intekglobal?'
    mock_input.side_effect = [question, 'Y', 'N', new_answer, question, 'N']

    data = {'context': defaultdict(str), 'faq': defaultdict(str)}
    contexts = {
        'context':
            [
                '''One of the greatest punk rock bands from all the time
                is the Ramones.
                '''
            ],
        'topic': ['Ramones']
    }

    faqs = {
        'Question':
            ['Who was the leading vocal artist of the Ramones?', 'Is this is heaven?'],
        'Answer': ['Joey Ramone', 'No, it is life on earth']
    }

    with tempfile.TemporaryDirectory() as tmpdirname:

        mock_faq_files(tmpdirname, faqs, data['faq'])
        mock_context_file(tmpdirname, contexts, data['context'])

        qa_models = load_qa_models(
            config_tfidf=data['faq']['config'], download=False
        )
        old_responses, old_status = question_response(data, qa_models)
        new_responses, new_status = question_response(data, qa_models)

        logging.debug(f'Old response:\n {old_responses}')
        logging.debug(f'New response:\n{new_responses}')
        
        new_answer not in old_responses
        new_answer in new_responses

#test_updated_faq_response()

In [ ]:
# Test Contex Dialog System's part

In [ ]:
import tempfile
from unittest.mock import patch
from shutil import copyfile
from collections import defaultdict


def mock_faq_files(tmpdirname, faqs, faq_dic):

    faq_dic['path'] = path.join(tmpdirname, 'temp_faq.csv')
    faq_dic['config'] = path.join(tmpdirname, 'temp_config_faq.json')
    faq_dic['df'] = pd.DataFrame(faqs)
    faq_dic['df'].to_csv(faq_dic['path'], index=False)

    copyfile(configs.faq.tfidf_logreg_en_faq, faq_dic['config'])

    updates_faq_config_file(
        configs_path=faq_dic['config'],
        dataset_reader={'data_path': faq_dic['path']}
    )


def mock_context_file(tmpdirname, contexts, context_dic):

    context_dic['path'] = path.join(tmpdirname, 'temp_context.csv')
    context_dic['df'] = pd.DataFrame(contexts)
    context_dic['df'].to_csv(context_dic['path'], index=False)


@patch('__main__.get_input')
def test_updated_context_response(mock_input):
    data = {'context': defaultdict(str), 'faq': defaultdict(str)}

    contexts = {
        'context': ['Intekglobal has its headquarters located in TJ', ],
        'topic': ['headquarters']
    }

    faqs = {
        'Question':
            ['Minimum number of questions?', 'This is the other question?'],
        'Answer': ['Two', 'yes']
    }

    question = 'What is a Chatbot?'
    new_topic = 'AI Tool & Chatbot Development'
    new_context = '''

A chatbot is an important tool for simulating intelligent conversations with humans.
Intekglobal chatbots efficiently live message on platforms such as Facebook Messenger, 
Slack, and Telegram. Assisting consumers with a variety of purposes and industries. 

But chatbots are more than just a cool technology advancement. They actually transform the user experience.
People want simple and convenient interactions with interface and products.

'''

    mock_input.side_effect = [
        question, 'YES', 'yes', new_topic, new_context, question, 'N'
    ]

    with tempfile.TemporaryDirectory() as tmpdirname:

        mock_faq_files(tmpdirname, faqs, data['faq'])
        mock_context_file(tmpdirname, contexts, data['context'])

        qa_models = load_qa_models(
            config_tfidf=data['faq']['config'], download=False
        )

        old_responses, old_status = question_response(data, qa_models)
        new_responses, new_status = question_response(data, qa_models)

        logging.info(f'Old status: {old_status}')
        logging.info(f'Old response:\n {old_responses}')
        logging.info(f'New response:\n{new_responses}')

        updated_context = pd.read_csv(data['context']['path'])

        assert updated_context[updated_context['context'] == new_context
                              ].shape[0] == 1

        assert updated_context[updated_context['topic'] == new_topic
                              ].shape[0] == 1

        assert 'an important tool for simulating intelligent conversations with humans' not in old_responses
        assert 'an important tool for simulating intelligent conversations with humans' in new_responses


#test_updated_context_response()

In [ ]:
#export
def dialog_system(context_data_file=None, faq_data_file=None, configs_faq=None):
    '''
     Main Dialog System
    '''
    run_shell_installs()

    data = {'context': defaultdict(str), 'faq': defaultdict(str)}

    load_and_prepare_data(
        context_data_file=context_data_file,
        faq_data_file=faq_data_file,
        configs_faq=configs_faq,
        data=data
    )

    qa_models = load_qa_models(config_tfidf=data['faq']['config'])

    while True:
        try:
            responses, _ = question_response(data=data, qa_models=qa_models)
        except (KeyboardInterrupt, EOFError, SystemExit):
            logging.debug('Goodbye!')
            return 'Goodbye!'

In [ ]:
# test  dialog_system()

from unittest.mock import patch
from let_me_answer_for_you.dialog_system import load_qa_models, load_and_prepare_data
from let_me_answer_for_you.dialog_system import question_response,run_shell_installs

@patch('__main__.run_shell_installs')
@patch('__main__.load_qa_models')
@patch('__main__.load_and_prepare_data')
@patch('__main__.question_response')
def test_main_keyboard_interrupt(
    mock_question_response,
    mock_pd_read_csv,
    mock_load_qa_models,
    mock_run_shell_installs,
):
    mock_question_response.side_effect = [
        KeyboardInterrupt(), EOFError(),
        SystemExit()
    ]
    assert 'Goodbye!' == dialog_system()
    assert 'Goodbye!' == dialog_system()
    assert 'Goodbye!' == dialog_system()


test_main_keyboard_interrupt()